**Fed-Ex Exploratory Data Analysis (EDA)**

##**Project Summary:**
This project analyzes the Fed-Ex dataset to evaluate delivery performance across various dimensions such as managing teams, shipment modes, countries of origin, and INCO terms. Key metrics like on-time delivery rate, delivery delays, lead times, and insurance costs are explored through data cleaning, transformation, and visualizations using Python.

Project Type: EDA

Contribution: Individual

Author:Mohamed Arshad Ahamed T

##GitHub Link:https://github.com/Ahmed-025/FedEx-EDA

##**Problem Statement:**
The objective of this project is to analyze the SCMS Delivery History dataset to identify key factors influencing delivery performance, delays, and logistics costs. By cleaning, transforming, and visualizing the data, the goal is to uncover patterns in shipment behavior across countries, shipment modes, managing teams, and vendor terms—ultimately supporting data-driven decisions to improve supply chain efficiency and on-time delivery rates.

##General Guidelines:


1. Data Cleaning & Preprocessing:

Remove irrelevant columns and duplicate entries.

Handle missing values appropriately, in fields like Shipment Mode, Weight, Insurance, and date columns.

Convert necessary columns to proper data types (datetime, string, numeric).

2. Categorical Grouping:

Consolidate less frequent or miscellaneous entries under unified categories ( combining certain values under "Others").

3. Exploratory Data Analysis (EDA):

Use bar charts and pie charts to analyze:

Shipments by country

On-time delivery rates by managing team and shipment mode

Delay trends by country

Freight cost and insurance by mode and weight

Use scatter plots and histograms for relationships and distributions, such as:

Lead time vs. on-time delivery

Weight vs. insurance cost

Delay distribution

4. Trend Analysis:

Analyze how shipments and freight costs vary over time (monthly/yearly).

Create time series charts to identify seasonal or performance trends.

5. Visualization:

Use meaningful labels, titles, and legends.

Choose appropriate color palettes for better interpretation.

##Lets Begin !

Import Dataset

In [ ]:
from google.colab import files
uploaded = files.upload()

Dataset Loading

In [ ]:
import pandas as pd

df = pd.read_csv("SCMS_Delivery_History_Dataset.csv")

Dataset First View

In [ ]:
# Check the first few rows
df.head()

Dataset Rows and Coumns count

In [ ]:
df.shape

Dataset Information

In [ ]:
df.info()

Dataset Columns

In [ ]:
df.columns

Drop certain columns

In [ ]:
df = df.drop(['PQ #','PO / SO #','ASN/DN #','PQ First Sent to Client Date','Item Description','Molecule/Test Type','Dosage','Dosage Form'], axis = 1)
df.head()

Duplicate Values

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
# Convert object columns to string (for categorical/text analysis)
text_cols = ['Country', 'Managed By', 'Fulfill Via']

for col in text_cols:
    df[col] = df[col].astype('string')

# Confirm conversion
print(df[text_cols].dtypes)

Missing/Null values

In [ ]:
df['Shipment Mode'].value_counts()

Handling Shipment mode null values

In [ ]:
df['Shipment Mode'] = df['Shipment Mode'].astype('string')
df['Shipment Mode'] = df['Shipment Mode'].fillna('Air')

Handling Weight (kilograms) column

In [ ]:
df['Weight (Kilograms)'].unique()

In [ ]:
df['Weight (Kilograms)'].value_counts()

In [ ]:
df['Weight (Kilograms)'].value_counts().loc['Weight Captured Separately']
#Should return 1507

In [ ]:
df['Weight (Kilograms)'] = pd.to_numeric(df['Weight (Kilograms)'], errors='coerce')

In [ ]:
mean_weight = df['Weight (Kilograms)'].mean()
print("Average Weight:", mean_weight)

In [ ]:
df['Weight (Kilograms)'] = df['Weight (Kilograms)'].fillna(mean_weight)

In [ ]:
df['Weight (Kilograms)'] = df['Weight (Kilograms)'].round().astype(int)
df['Weight (Kilograms)'].value_counts()

Handling Sub Classification column

In [ ]:
df['Sub Classification'].value_counts()

In [ ]:
df['Sub Classification'] = df['Sub Classification'].replace({
    'HIV test - Ancillary': 'Others',
    'Malaria': 'Others',
    'ACT': 'Others'
})

In [ ]:
df['Sub Classification'].value_counts()

Verifying other columns

In [ ]:
df['Brand'].unique()

In [ ]:
df['Unit of Measure (Per Pack)'].unique()

In [ ]:
df['Line Item Quantity'].value_counts()

In [ ]:
df['Freight Cost (USD)'].value_counts()

Handling Freight Cost (USD) column

In [ ]:
df['Freight Cost (USD)'] = df['Freight Cost (USD)'].replace({
    'Freight Included in Commodity Cost': 0,
    'Invoiced Separately': 0
})

In [ ]:
df['Freight Cost (USD)'] = pd.to_numeric(df['Freight Cost (USD)'], errors='coerce')
mean_freight = df['Freight Cost (USD)'].mean()
df['Freight Cost (USD)'] = df['Freight Cost (USD)'].fillna(mean_freight)

In [ ]:
df['Freight Cost (USD)'] = df['Freight Cost (USD)'].round().astype(int)

Handling PO send to Vendor Date column

In [ ]:
df['PO Sent to Vendor Date'].value_counts()

In [ ]:
df['PO Sent to Vendor Date'] = df['PO Sent to Vendor Date'].replace(
    ['N/A - From RDC', 'Date Not Captured'],
    pd.NaT
)

In [ ]:
df['Delivery Date'] = pd.to_datetime(df['PO Sent to Vendor Date'])
# Get the earliest delivery date
earliest_date = df['Delivery Date'].min()

# Print the result
print("Earliest Delivery Date:", earliest_date.date())

In [ ]:
df['PO Sent to Vendor Date'] = df['PO Sent to Vendor Date'].fillna(pd.to_datetime('2007-02-07'))

Handling Life term Insurance column

In [ ]:
print(df['Line Item Insurance (USD)'].isna().sum())

In [ ]:
df = df.dropna(subset=['Line Item Insurance (USD)'])

Remaining number of rows

In [ ]:
print(f"Remaining rows: {len(df)}")


Dataset Rows and Columns

In [ ]:
df.head()

##New Columns for Analysis

1. On-Time Delivery: delivered on or before scheduled date

In [ ]:
df['On Time Delivery'] = (df['Delivered to Client Date'] <= df['Scheduled Delivery Date']).astype(int)

2. Delivery Delay in Days

In [ ]:
df['Delivery Delay (days)'] = (df['Delivered to Client Date'] - df['Scheduled Delivery Date']).dt.days

3. Lead Time in Days: time between PO sent and scheduled delivery

In [ ]:
df['Lead Time (days)'] = (df['Scheduled Delivery Date'] - df['PO Sent to Vendor Date']).dt.days

##Visualization

1. Number of Shipments by Country

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
top_countries = df['Country'].value_counts().head(15)

plt.figure(figsize=(10, 6))
sns.barplot(x=top_countries.values, y=top_countries.index, palette='viridis')
plt.title('Top 15 Countries by Number of Shipments')
plt.xlabel('Number of Shipments')
plt.ylabel('Country')
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.show()

2. Shipment Mode Distribution

In [ ]:
shipment_mode_counts = df['Shipment Mode'].value_counts()

plt.figure(figsize=(7, 7))
plt.pie(shipment_mode_counts.values, labels=shipment_mode_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Shipment Mode Distribution')
plt.tight_layout()
plt.show()

3. Average Freight Cost by Shipment Mode

In [ ]:
avg_freight_by_mode = df.groupby('Shipment Mode')['Freight Cost (USD)'].mean().sort_values(ascending=False)

plt.figure(figsize=(8, 5))
sns.barplot(x=avg_freight_by_mode.index, y=avg_freight_by_mode.values, palette='coolwarm')
plt.title('Average Freight Cost by Shipment Mode')
plt.ylabel('Average Freight Cost (USD)')
plt.xlabel('Shipment Mode')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

4. Distribution of Lead Time (in Days)

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df['Lead Time (days)'], bins=30, kde=True, color='teal')
plt.title('Distribution of Lead Time (Days)')
plt.xlabel('Lead Time (Days)')
plt.ylabel('Frequency')
plt.tight_layout()
plt.show()

5. Boxplot of Insurance Cost by Shipment Mode

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Shipment Mode', y='Line Item Insurance (USD)', palette='Pastel1')
plt.title('Insurance Cost by Shipment Mode')
plt.xlabel('Shipment Mode')
plt.ylabel('Insurance Cost (USD)')
plt.xticks(rotation=45)
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

6. Are shipments managed by specific teams (e.g., PMO-US) more likely to be delivered on time?

In [ ]:
on_time_by_team = df.groupby('Managed By')['On Time Delivery'].mean().sort_values(ascending=False)

sns.barplot(x=on_time_by_team.values, y=on_time_by_team.index)
plt.title('On-Time Delivery Rate by Managing Team')
plt.xlabel('On-Time Delivery Rate')
plt.ylabel('Managed By')
plt.show()

7. Does shipment mode impact on-time delivery likelihood?

In [ ]:
on_time_by_mode = df.groupby('Shipment Mode')['On Time Delivery'].mean().sort_values(ascending=False)

sns.barplot(x=on_time_by_mode.values, y=on_time_by_mode.index, palette='Blues')
plt.title('On-Time Delivery Rate by Shipment Mode')
plt.xlabel('On-Time Delivery Rate')
plt.ylabel('Shipment Mode')
plt.show()

8. Do shipments from certain countries experience more delays?

In [ ]:
avg_delay_by_country = df.groupby('Country')['Delivery Delay (days)'].mean().sort_values(ascending=False).head(20)
plt.figure(figsize=(10, 7))
sns.barplot(
    x=avg_delay_by_country.values,
    y=avg_delay_by_country.index,
    palette='Blues_r'  # use a consistent blue scale
)
plt.title('Top 20 Countries by Average Delivery Delay')
plt.xlabel('Average Delay (Days)')
plt.ylabel('Country')
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

9. Does shipment mode influence the frequency of late deliveries?

In [1]:
late_shipments_by_mode = df[df['On Time Delivery'] == 0]['Shipment Mode'].value_counts()

sns.barplot(x=late_shipments_by_mode.index, y=late_shipments_by_mode.values, palette='Reds')
plt.title('Late Deliveries by Shipment Mode')
plt.xlabel('Shipment Mode')
plt.ylabel('Number of Late Deliveries')
plt.xticks(rotation=45)
plt.show()

NameError: name 'df' is not defined

10. Does more lead time (between PO sent and scheduled delivery) help on-time delivery?

In [ ]:
sns.scatterplot(x='Lead Time (days)', y='On Time Delivery', data=df, alpha=0.3)
plt.title('Lead Time vs On-Time Delivery')
plt.xlabel('Lead Time (Days)')
plt.ylabel('On Time (1 = Yes, 0 = No)')
plt.show()

# Correlation value
lead_time_corr = df[['Lead Time (days)', 'On Time Delivery']].corr().iloc[0,1]
print(f"Correlation between Lead Time and On-Time Delivery: {lead_time_corr:.2f}")
#df = df[df['Lead Time (days)'] >= 0]

11. Does the type of INCO term impact delivery performance?

In [ ]:
on_time_by_inco = df.groupby('Vendor INCO Term')['On Time Delivery'].mean().sort_values(ascending=False)

sns.barplot(x=on_time_by_inco.values, y=on_time_by_inco.index, palette='Set2')
plt.title('On-Time Delivery Rate by INCO Term')
plt.xlabel('On-Time Delivery Rate')
plt.ylabel('Vendor INCO Term')
plt.show()

12. Are higher weights linked to higher insurance costs?

In [ ]:
sns.scatterplot(x='Weight (Kilograms)', y='Line Item Insurance (USD)', data=df, alpha=0.3)
plt.title('Weight vs Insurance Cost')
plt.xlabel('Weight (kg)')
plt.ylabel('Insurance Cost (USD)')
plt.show()

# Correlation value
insurance_corr = df[['Weight (Kilograms)', 'Line Item Insurance (USD)']].corr().iloc[0,1]
print(f"Correlation between Weight and Insurance Cost: {insurance_corr:.2f}")

**Conclusion:**
The analysis of the Fed-Ex dataset reveals several key insights into supply chain performance. Shipments managed by specific teams (like PMO-US) and sent via faster modes (like Air) tend to have higher on-time delivery rates. Countries of origin and shipment modes significantly impact delivery delays, highlighting the importance of regional planning and logistics strategy. Insurance costs show a positive relationship with shipment weight, underlining the need to monitor heavy shipments closely. These findings provide actionable data to enhance operational efficiency, reduce delays, and control logistics costs in global supply chain management.